- [ ] Add colab button

# Deep Learning - Dr. Tristan Behrens
## Today: End to End Learning for Self-Driving Cars.

<img src="https://ai-guru.s3.eu-central-1.amazonaws.com/FHWS/demo.gif" alt="Drawing" style="width:400px; margin-left:0" />

Paper: https://arxiv.org/abs/1604.07316  
Dataset: https://www.kaggle.com/roydatascience/training-car

What is the paper about?

- Nvidia.
- Training CNN on raw pixels.
- One camera mapped to steering commands.
- 72 hours of training data from two cars. ???
- No explicit decomposition of the problem.


## 1. Import modules.

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tensorflow.keras import models, layers, optimizers
from tqdm import tqdm

## 2. Data.

### 2.1. Download datasets.

In [ ]:
unprocessed_data_path = "training-car-unprocessed.p"
data_path = "training-car.p"

if os.path.exists(unprocessed_data_path) == False:
    !wget https://ai-guru.s3.eu-central-1.amazonaws.com/FHWS/training-car-unprocessed.p
else:
    print("Data already downloaded.")
        
if os.path.exists(data_path) == False:
    !wget https://ai-guru.s3.eu-central-1.amazonaws.com/FHWS/training-car.p
else:
    print("Data already downloaded.")
    

### 2.2. Load and inspect unprocessed data.

Firstly, we define a method for rendering random samples from any dataset.

In [ ]:
def render_random_samples(images, targets):

    for i in range(3):

        _, _ = plt.subplots(1, 3, figsize=(15,15))

        subplot_index = 1
        for j in np.random.choice(list(range(len(images))), 3):

            image = images[j] / 255.0
            target = targets[j]
            plt.subplot(1, 3, subplot_index)
            plt.imshow(image, cmap="gray")
            plt.title("Steering {:0.2f}".format(target))
            subplot_index += 1
        plt.show()
        plt.close()

---
Then we load the unpropressed data. Note: Not used for training.

In [ ]:
# Load pickle file.
with open(unprocessed_data_path, "rb") as file:
    images_unprocessed, targets_unprocessed = pickle.load(file)
print(images_unprocessed.shape)
print(targets_unprocessed.shape)

render_random_samples(images_unprocessed, targets_unprocessed)

### 2.3. About data preprocessing.

Steps:
1. Load image.
2. Convert to [HSV](https://en.wikipedia.org/wiki/HSL_and_HSV).
3. Discard H and S. Keep only V.
4. Resize to 40x40.

Preprocessing happened offline. See preprocessing notebook for details. Try at home.

**Questions**: 
- Why preprocessing?
- Especially: Why downscaling?

### 2.4. Load and inspect preprocessed data.

This is the data we will use for training.

In [ ]:
with open(data_path, "rb") as file:
    images, targets = pickle.load(file)
print(images.shape)
print(targets.shape)

render_random_samples(images, targets)

### 2.5 Render distribution of targets.

In [ ]:
print("Min:", np.min(targets))
print("Mean:", np.mean(targets))
print("STD:", np.std(targets))
print("Max:", np.max(targets))

plt.figure(figsize=(10, 4))
plt.hist(targets, bins=75)
plt.show()
plt.close()

**Questions:**
- Is there a problem with this distribution?
- Is there a problem with the dataset size?

## 3. Designing and training a Neural Network.

### 3.1. Neural Network architecture.

- Lambda: Apply a function.
- Reshape: Change dimensions.
- Conv2D: Applying multiple filters on all possible locations.
- MaxPooling2D: Downsampling with maximum function.
- Flatten: Go down to one dimension.
- Dropout: Randomly set inputs to zero.
- BatchNormalization: Normalize inputs using mean and STD.
- Dense: Fully-connected.

In [ ]:
model = models.Sequential()

model.add(layers.Lambda(lambda image: image / 127.5 - 1., input_shape=(40, 40)))
model.add(layers.Reshape((40, 40, 1)))

model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D((2, 2), padding='valid'))

model.add(layers.Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(layers.Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D((2, 2), padding="valid"))

model.add(layers.Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D((2, 2), padding="valid"))

model.add(layers.Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(layers.MaxPooling2D((2, 2), padding="valid"))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))

model.add(layers.BatchNormalization())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(1, activation="linear"))

model.summary()

---
Compilation adds optimizer, loss and metrics.

- Nadam: Adam with Nesterov momentum.
- MSE: Mean squared error.
- MAE: Mean absolute error.

In [ ]:
model.compile(
    optimizer=optimizers.Nadam(lr=0.0001), 
    loss="mse",
    metrics=["mae"])

### 3.2. Evaluate model quality (before training).

Render a random image, the target, the prediction and the error.

In [ ]:
def render_sample():
    random_index = random.randint(0, len(images) - 1)
    image = images[random_index]
    target = targets[random_index]
    predicted_target = model.predict(np.array([image]))[0][0]
    plt.imshow(image, cmap="gray")
    error = np.abs(target - predicted_target)
    plt.title("Predicted: {:0.2f} Expected: {:0.2f} Error: {:0.2f}".format(predicted_target, target, error))

In [ ]:
render_sample()

---

Get the error(s) for the entire dataset.

In [ ]:
mse, mae = model.evaluate(images, targets)
print("MSE: {:0.2f}".format(mse))
print("MAE: {:0.2f}".format(mae))

---
Visualize how well the model predicts.

In [ ]:
def render_predictions():
    expected_targets = []
    predicted_targets = []
    for i in tqdm(range(0, 1000, 3)):
        image = images[i]
        expected_target = targets[i]
        predicted_target = model.predict(np.array([image]))[0][0]
        expected_targets.append(expected_target)
        predicted_targets.append(predicted_target)

    plt.figure(figsize=(10, 5))
    axes = plt.gca()
    axes.set_ylim([-0.5, 0.5])
    plt.plot(expected_targets, label="Expected")
    plt.plot(predicted_targets, label="Predicted")
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
render_predictions()

### 3.3. Train the model.

In [ ]:
history = model.fit(
    images, targets,
    epochs=10,
    batch_size=128,
    validation_split=0.2
)

---
Render the training history.

In [ ]:
# MSE.
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.show()
plt.close()

# MAE.
if "mae" in history.history:
    plt.plot(history.history["mae"], label="mae")
    plt.plot(history.history["val_mae"], label="val_mae")
    plt.legend()
    plt.show()
    plt.close()
elif "mean_absolute_error" in history.history:
    plt.plot(history.history["mean_absolute_error"], label="mean_absolute_error")
    plt.plot(history.history["val_mean_absolute_error"], label="val_mean_absolute_error")
    plt.legend()
    plt.show()
    plt.close()  

**Questions:**
- What do we see here?
- How good is the Neural Network?
- Which plot is the most important one?

### 3.4. Evaluate model quality (after training).

Render a random image, the target, the prediction and the error.

In [ ]:
render_sample()

---

Get the error(s) for the entire dataset.

In [ ]:
mse, mae = model.evaluate(images, targets)
print("MSE: {:0.2f}".format(mse))
print("MAE: {:0.2f}".format(mae))

---
Visualize how well the model predicts.

In [ ]:
render_predictions()

## 4. Summary and outlook.

Summary:

- Convolutional Neural Networks can solve Autonomous Driving.
- Input comes from camera, output is steering.
- A lot of data is necessary.
    
Outlook:

- Train on way more data.
- Have a closer look at how convolutions and pooling-layers work.
- Train on more steering, throttle and brake.
- [Autonomous Drone Navigation with Deep Learning](https://www.youtube.com/watch?v=H7Ym3DMSGms).

## References.

- https://www.manning.com/books/deep-learning-with-python
- https://www.deeplearningbook.org